<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Monte-Carlo-Simulation" data-toc-modified-id="Monte-Carlo-Simulation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Monte Carlo Simulation</a></span></li><li><span><a href="#Scipy-Optimize---Minimize" data-toc-modified-id="Scipy-Optimize---Minimize-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Scipy Optimize - Minimize</a></span></li></ul></div>

# Comparing methods of optimization

In [1]:
import pandas_datareader.data as web
import numpy as np, pandas as pd, scipy as sp
import scipy.optimize as sco
from scipy import stats
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
%matplotlib inline
p = print

In [2]:
# Load the data
df = pd.read_csv('AAPL_MSFT_IBM_WMT.csv', index_col='Date', parse_dates=True)

In [3]:
# Make a list of stock symbols
tickers = df.columns

In [4]:
# Print the stock symbols
tickers

Index(['AAPL', 'IBM', 'MSFT', 'WMT'], dtype='object')

In [5]:
# Calculate the daily returns
returns = df.pct_change().dropna()

In [6]:
# Calculate the mean and standard deviation of returns
mean_return = returns.mean()
return_stdev = returns.std()

In [7]:
# Calculate the covariance of returns
cov_matrix = returns.cov()

In [8]:
# Hypothetical risk-free rate of retrun
rf = 0.00003

## Monte Carlo Simulation

In [9]:
def simulate_random_portfolios(num_portfolios, returns, rf):
    # Initialize simulatio results to all zeros
    results_matrix = np.zeros((len(tickers)+3, num_portfolios))
    
    for i in range(num_portfolios):
        # Generate random weights
        weights = np.random.random(len(tickers))
        # Normalize the weights and set the sum to 1
        weights /= np.sum(weights)
        
        # Calculate the portfolio return, standard deviation, and Sharpe ratio
        portfolio_return = np.sum(weights * returns.mean()) * 252
        portfolio_stdev = np.sqrt(weights.T.dot(returns.cov()*252).dot(weights))
        portfolio_sharpe_ratio = (portfolio_return - rf) / portfolio_stdev
        
        # Store the results in the arrays
        results_matrix[0,i] = portfolio_return
        results_matrix[1,i] = portfolio_stdev
        results_matrix[2,i] = portfolio_sharpe_ratio
        
        # Save the weights in the array
        for j in range(len(weights)):
            results_matrix[j+3,i] = weights[j]
    
    # Save the simulations in a dataframe
    results_df = pd.DataFrame(results_matrix.T,columns=['ret','stdev','sharpe'] + [ticker for ticker in tickers])
     
    return results_df

In [10]:
results = simulate_random_portfolios(num_portfolios=100000, returns=returns,rf = 0.00003)

In [11]:
results.head()

,ret,stdev,sharpe,AAPL,IBM,MSFT,WMT
0,0.108333,0.144928,0.747288,0.039125,0.382157,0.108706,0.470012
1,0.165769,0.165305,1.002626,0.309328,0.268325,0.263885,0.158462
2,0.185738,0.168850,1.099838,0.461451,0.112016,0.123440,0.303092
3,0.126709,0.168651,0.751133,0.013002,0.437615,0.421935,0.127447
4,0.172092,0.163159,1.054567,0.114530,0.055101,0.484206,0.346162


In [12]:
max_sharpe_port = results.iloc[results['sharpe'].idxmax()]
p(max_sharpe_port)

ret       0.196744
stdev     0.168566
sharpe    1.166985
AAPL      0.391849
IBM       0.000019
MSFT      0.278419
WMT       0.329713
Name: 74480, dtype: float64


In [13]:
weights_max_sharpe_port = max_sharpe_port[3:]
weights_max_sharpe_port

AAPL    0.391849
IBM     0.000019
MSFT    0.278419
WMT     0.329713
Name: 74480, dtype: float64

In [14]:
min_vol_port = results.iloc[results['stdev'].idxmin()]
min_vol_port

ret       0.122449
stdev     0.143459
sharpe    0.853332
AAPL      0.095986
IBM       0.283111
MSFT      0.102949
WMT       0.517954
Name: 69578, dtype: float64

In [15]:
weights_min_vol_port = min_vol_port[3:]
weights_min_vol_port

AAPL    0.095986
IBM     0.283111
MSFT    0.102949
WMT     0.517954
Name: 69578, dtype: float64

## Scipy Optimize - Minimize

In [16]:
# Calculate Max Sharpe Ration pf with Scipy Optimize function
def calc_neg_sharpe(weights, mean_return, cov_matrix, rf):
    portfolio_return = np.sum(weights * returns.mean()) * 252
    portfolio_stdev = np.sqrt(weights.T.dot(returns.cov() * 252).dot(weights))
    sharpe_ratio = (portfolio_return - rf) / portfolio_stdev
    return -1 * sharpe_ratio

In [17]:
def max_sharpe_ratio(mean_return, cov_matrix, rf=0.00003):
    num_assets = len(mean_return)
    args = (mean_return, cov_matrix, rf)
    constraints = ({'type':'eq', 'fun':lambda x: np.sum(x) - 1})
    bound = (0.0, 1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(calc_neg_sharpe, num_assets * [1./num_assets],
                           args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [18]:
# max_sharpe_ratio(mean_return, cov_matrix, rf)

In [19]:
max_sharpe_pf = max_sharpe_ratio(mean_return, cov_matrix, rf)

In [20]:
weights_max_sharpe_pf = pd.DataFrame([x for x in max_sharpe_pf['x']], index=tickers).T

In [21]:
weights_max_sharpe_pf

,AAPL,IBM,MSFT,WMT
0,0.389181,0.0,0.279299,0.331519


In [22]:
# Calculate Min Volatility pf with Scipy Optimize function
def calc_pf_stdev(weights, mean_return, cov_matrix):
    portfolio_stdev = np.sqrt(weights.T.dot(returns.cov() * 252).dot(weights))
    return portfolio_stdev

In [23]:
def min_vol(mean_return, cov_matrix):
    num_assets = len(mean_return)
    args = (mean_return, cov_matrix)
    constraints = ({'type':'eq', 'fun':lambda x: np.sum(x) - 1})
    bound = (0.0, 1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(calc_pf_stdev, num_assets*[1./num_assets],
                         args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [24]:
# min_vol(mean_return, cov_matrix)

In [25]:
min_vol_pf = min_vol(mean_return, cov_matrix)

In [26]:
weights_min_vol_pf = pd.DataFrame(data=[x for x in min_vol_pf['x']], index=tickers).T

In [27]:
weights_min_vol_pf

,AAPL,IBM,MSFT,WMT
0,0.106555,0.286021,0.09797,0.509455
